# Introduction - Using COSINE Metric

In this notebook we demonstrate the use of **LSI (Latent Semantic Indexing)** technique of Information Retrieval context to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each use case content as an entire document that must be returned to the query made

## Import Libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from utils import plots
from utils import firefox_dataset as fd
from utils import tokenizers as tok
from utils import aux_functions
from utils import model_evaluator as m_eval

from models.lsi import LSI
from models.model_hyperps import LSI_Model_Hyperp

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from IPython.display import display

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [3]:
test_cases_df = fd.read_testcases_df()
bug_reports_df = fd.read_bugreports_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.read_trace_df()
orc.set_index('tc_name', inplace=True, drop=True)

print(bug_reports_df.shape)
print(test_cases_df.shape)
print(orc.shape)

(35314, 12)
(207, 10)
(207, 35314)


## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

### Analysis with Default Values of LSI Model

In [12]:
%%time

best_model = LSI()
best_model.recover_links(corpus, query, test_cases_names, bug_reports_names)
evaluator = m_eval.ModelEvaluator(orc, best_model)
evaluator.evaluate_model(verbose=True)
#evaluator.plot_precision_vs_recall()

NameError: name 'pd' is not defined

## Running LSI Model with Different Types of Oracles

### Strong and Weak Links Datasets

In [9]:
br_tc_strong_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Strong.csv')
br_tc_weak_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Weak.csv')
br_tc_mix_df = pd.read_csv('../../data/mozilla_firefox_v2/firefoxDataset/oracle/output/BR_TC_Mix.csv')

print(br_tc_strong_df.shape)
print(br_tc_weak_df.shape)
print(br_tc_mix_df.shape)

(10, 4)
(10, 4)
(10, 4)


### Define **run_lsi_model()** Function

In [11]:
def run_lsi_model(selected_tcs, selected_brs):
    tcs_df = test_cases_df[test_cases_df.tc_name.isin(selected_tcs)]
    brs_df = bug_reports_df[bug_reports_df.br_name.isin(selected_brs)]

    corpus_subset = tcs_df.tc_desc
    query_subset = brs_df.br_desc
    testcases_names_subset = tcs_df.tc_name
    bug_reports_names_subset = brs_df.br_name
    orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

    print('TestCases Subset Shape: {}'.format(tcs_df.shape))
    print('BugReports Subset Shape: {}'.format(brs_df.shape))
    print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

    lsi_hyperp = {
        LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : ('cosine' , .80),
        LSI_Model_Hyperp.TOP.value : 10,
        LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: 50,
        LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: (1,1),
        LSI_Model_Hyperp.VECTORIZER.value : TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True),
        LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : tok.WordNetBased_LemmaTokenizer()
    }

    lsi_model = LSI(**lsi_hyperp)
    lsi_model.set_name('LSI_Model_0')
    lsi_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)

    print("\nModel Evaluation -------------------------------------------")
    evaluator = m_eval.ModelEvaluator(orc_subset_df, lsi_model)
    evaluator.evaluate_model(verbose=True)

    print("\n\nTraceLinks Matrix --------------------------------------")
    display(aux_functions.highlight_df(lsi_model.get_trace_links_df()))
    
    print("\n\nOracle -----------------------------------------")
    display(aux_functions.highlight_df(orc_subset_df))

### Oracle with Strong Links Only

In [25]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_strong_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_strong_df.BR.values]

run_lsi_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (6, 12)
Oracle Subset Shape: (10, 6)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LSI_Model_0': 0.2916666666666667,
              'Mean Precision of LSI_Model_0': 1.0,
              'Mean Recall of LSI_Model_0': 0.17460317460317457},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 50,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'conten

br_name,BR_1298575_SRC,BR_1313805_SRC,BR_1320658_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,,
TC_118_TRG,0,0,0,0,0,0
TC_120_TRG,1,0,0,0,0,0
TC_121_TRG,0,0,0,0,0,0
TC_143_TRG,0,0,0,0,0,1
TC_155_TRG,0,1,0,0,0,0
TC_172_TRG,0,0,0,1,0,0
TC_181_TRG,0,0,0,0,0,0
TC_183_TRG,0,0,1,0,0,0
TC_196_TRG,0,0,0,0,0,0




Oracle -----------------------------------------


,BR_1298575_SRC,BR_1313805_SRC,BR_1320658_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,,
TC_118_TRG,1,0,0,0,0,0
TC_120_TRG,1,0,0,0,0,0
TC_121_TRG,1,0,0,0,0,0
TC_143_TRG,0,1,1,1,1,1
TC_155_TRG,0,1,1,1,1,1
TC_172_TRG,0,1,1,1,1,1
TC_181_TRG,0,1,1,1,1,1
TC_183_TRG,0,1,1,1,1,1
TC_196_TRG,0,1,1,1,1,1


### Oracle with Weak Links Only

In [17]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_weak_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_weak_df.BR.values]

run_lsi_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (5, 12)
Oracle Subset Shape: (10, 5)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LSI_Model_0': 0.1388888888888889,
              'Mean Precision of LSI_Model_0': 0.4,
              'Mean Recall of LSI_Model_0': 0.08571428571428572},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 50,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'conten

br_name,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,0,0,0,0,0
TC_75_TRG,0,0,0,0,0
TC_105_TRG,0,0,0,0,0
TC_154_TRG,0,0,0,0,0
TC_155_TRG,0,0,0,0,0
TC_174_TRG,0,0,0,0,0
TC_196_TRG,0,1,0,0,0
TC_197_TRG,0,0,0,1,0
TC_200_TRG,0,0,0,0,1




Oracle -----------------------------------------


,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,0,0,0,0
TC_75_TRG,0,1,0,0,0
TC_105_TRG,0,1,0,0,0
TC_154_TRG,0,0,1,1,1
TC_155_TRG,0,0,1,1,1
TC_174_TRG,0,0,1,1,1
TC_196_TRG,0,0,1,1,1
TC_197_TRG,0,0,1,1,1
TC_200_TRG,0,0,1,1,1


### Oracle with Mixed Links (Strong and Weak)

In [18]:
selected_tcs = ['TC_{}_TRG'.format(tc_num) for tc_num in br_tc_mix_df.TC.values]
selected_brs = ['BR_{}_SRC'.format(bg_num) for bg_num in br_tc_mix_df.BR.values]

run_lsi_model(selected_tcs, selected_brs)

TestCases Subset Shape: (10, 10)
BugReports Subset Shape: (5, 12)
Oracle Subset Shape: (10, 5)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LSI_Model_0': 0.27142857142857146,
              'Mean Precision of LSI_Model_0': 0.8,
              'Mean Recall of LSI_Model_0': 0.16666666666666666},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 50,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'conte

br_name,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,0,0,0,0,0
TC_105_TRG,0,0,0,0,0
TC_118_TRG,0,0,0,0,0
TC_121_TRG,0,1,0,0,0
TC_143_TRG,0,0,0,0,1
TC_154_TRG,0,0,0,0,0
TC_155_TRG,0,0,0,0,0
TC_172_TRG,0,0,1,0,0
TC_174_TRG,0,0,0,0,0




Oracle -----------------------------------------


,BR_1285719_SRC,BR_1298575_SRC,BR_1329292_SRC,BR_1329421_SRC,BR_1329430_SRC
tc_name,,,,,
TC_35_TRG,1,0,0,0,0
TC_105_TRG,0,1,0,0,0
TC_118_TRG,0,1,0,0,0
TC_121_TRG,0,1,0,0,0
TC_143_TRG,0,0,1,1,1
TC_154_TRG,0,0,1,1,1
TC_155_TRG,0,0,1,1,1
TC_172_TRG,0,0,1,1,1
TC_174_TRG,0,0,1,1,1


### General Test

In [14]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

selected_bugreports = bugreports_subset_df.br_name
run_lsi_model(selected_testcases, selected_bugreports)


TestCases Subset Shape: (6, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (6, 15)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of LSI_Model_0': 0.07619047619047618,
              'Mean Precision of LSI_Model_0': 0.26666666666666666,
              'Mean Recall of LSI_Model_0': 0.04444444444444444},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 50,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                          

br_name,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0
TC_14_TRG,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
TC_15_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
TC_17_TRG,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
TC_18_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0




Oracle -----------------------------------------


,BR_1268934_SRC,BR_1282551_SRC,BR_1291175_SRC,BR_1299787_SRC,BR_1418983_SRC,BR_1432520_SRC,BR_1436749_SRC,BR_1443632_SRC,BR_1443754_SRC,BR_1450216_SRC,BR_1461828_SRC,BR_1463274_SRC,BR_1463735_SRC,BR_1497738_SRC,BR_1513270_SRC
tc_name,,,,,,,,,,,,,,,
TC_13_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_14_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_17_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
TC_18_TRG,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0


### Backup Code

In [20]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

corpus_subset = tc_subset_df.tc_desc
query_subset = bugreports_subset_df.br_desc
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

lsi_hyperp = {
    LSI_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : ('cosine' , .80),
    LSI_Model_Hyperp.TOP.value : 10,
    LSI_Model_Hyperp.SVD_MODEL_N_COMPONENTS.value: 100,
    LSI_Model_Hyperp.VECTORIZER_NGRAM_RANGE.value: (1,1),
    LSI_Model_Hyperp.VECTORIZER.value : TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True),
    LSI_Model_Hyperp.VECTORIZER_TOKENIZER.value : tok.WordNetBased_LemmaTokenizer()
}

lsi_model = LSI(**lsi_hyperp)
lsi_model.set_name('LSI_Model_0')
lsi_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)

evaluator = m_eval.ModelEvaluator(orc_subset_df, lsi_model)
evaluator.evaluate_model(verbose=True)

display(aux_functions.highlight_df(lsi_model.get_trace_links_df()))
display(aux_functions.highlight_df(orc_subset_df))


TestCases Subset Shape: (14, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (14, 15)
{'Measures': {'Mean FScore of LSI_Model_0': 0.016666666666666666,
              'Mean Precision of LSI_Model_0': 0.06666666666666667,
              'Mean Recall of LSI_Model_0': 0.009523809523809523},
 'Setup': [{'Name': 'LSI_Model_0'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.8)},
           {'Top Value': 10},
           {'SVD Model': {'algorithm': 'randomized',
                          'n_components': 100,
                          'n_iter': 10,
                          'random_state': 42,
                          'tol': 0.0}},
           {'Vectorizer': {'analyzer': 'word',
                           'binary': False,
                           'decode_error': 'strict',
                           'dtype': <class 'numpy.float64'>,
                           'encoding': 'utf-8',
                           'input': 'content',
                           'lowercas

br_name,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_165_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_152_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_160_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_169_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_145_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_149_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,BR_1268934_SRC,BR_1463735_SRC,BR_1432520_SRC,BR_1497738_SRC,BR_1463274_SRC,BR_1461828_SRC,BR_1443754_SRC,BR_1443632_SRC,BR_1282551_SRC,BR_1418983_SRC,BR_1291175_SRC,BR_1513270_SRC,BR_1450216_SRC,BR_1436749_SRC,BR_1299787_SRC
tc_name,,,,,,,,,,,,,,,
TC_165_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_19_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_152_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_16_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_160_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_169_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_145_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TC_15_TRG,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
TC_149_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
